# Intel-Bildklassifizierung (CNN – Keras)

Hallo, ich hoffe, Sie haben einen schönen Tag.

In diesem Notizbuch werde ich den Prozess der Implementierung von CNN mit Keras ausprobieren, um Bilder zu klassifizieren.
1. Zuerst importieren wir nützliche Pakete.
1. Anschließend laden wir die Daten, bevor wir sie visualisieren und vorverarbeiten.
1. Wir testen ein einfaches CNN-Modell und bewerten dann seine Leistung.
1. Anschließend werden wir ein vorab trainiertes Modell verwenden, um auch dieser Herausforderung zu begegnen.**

# # Pakete importieren

In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm

In [ ]:
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (150, 150)

# Laden der Daten
Wir müssen eine Load_Data-Funktion schreiben, die die Bilder und Beschriftungen aus dem Ordner lädt.

In [ ]:
def load_data():
    """
        Load the data:
            - 14,034 images to train the network.
            - 3,000 images to evaluate how accurately the network learned to classify images.
    """
    
    datasets = ['/kaggle/input/intel-image-classification/seg_train/seg_train', '/kaggle/input/intel-image-classification/seg_test/seg_test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

# Lassen Sie uns den Datensatz untersuchen
Wir können uns fragen:
* Wie viele Trainings- und Testbeispiele haben wir?
* Wie groß sind die Bilder?
* Wie groß ist der Anteil jeder beobachteten Kategorie?

In [ ]:
n_train = train_labels.shape[0]
n_test = test_labels.shape[0]

print ("Number of training examples: {}".format(n_train))
print ("Number of testing examples: {}".format(n_test))
print ("Each image is of size: {}".format(IMAGE_SIZE))

In [ ]:
import pandas as pd

_, train_counts = np.unique(train_labels, return_counts=True)
_, test_counts = np.unique(test_labels, return_counts=True)
pd.DataFrame({'train': train_counts,
                    'test': test_counts}, 
             index=class_names
            ).plot.bar()
plt.show()

In [ ]:
plt.pie(train_counts,
        explode=(0, 0, 0, 0, 0, 0) , 
        labels=class_names,
        autopct='%1.1f%%')
plt.axis('equal')
plt.title('Anteil jeder beobachteten Kategorie')
plt.show()

## Gute Praxis: Skalieren Sie die Daten

In [ ]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

## Visualisieren Sie die Daten
Wir können ein zufälliges Bild aus dem Trainingssatz anzeigen.

In [ ]:
def display_random_image(class_names, images, labels):
    """
        Display a random image from the images array and its correspond label from the labels array.
    """
    
    index = np.random.randint(images.shape[0])
    plt.figure()
    plt.imshow(images[index])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title('Image #{} : '.format(index) + class_names[labels[index]])
    plt.show()

In [ ]:
display_random_image(class_names, train_images, train_labels)

Wir können die ersten 25 Bilder aus dem Trainingssatz auch direkt mit einer Schleife anzeigen, um eine bessere Ansicht zu erhalten

In [ ]:
def display_examples(class_names, images, labels):
    """
        Zeigen Sie 25 Bilder aus dem Bilder-Array mit den entsprechenden Beschriftungen an
    """
    
    fig = plt.figure(figsize=(10,10))
    fig.suptitle("Zeigen Sie 25 Bilder aus dem Bilder-Array mit den entsprechenden Beschriftungen an", fontsize=16)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[labels[i]])
    plt.show()

In [ ]:
display_examples(class_names, train_images, train_labels)

# Anfänger: Einfache Modellerstellung

Schritte sind:
1. Bauen Sie das Modell,
1. Kompilieren Sie das Modell.
1. Trainieren / Anpassen der Daten an das Modell,
1. Bewerten Sie das Modell anhand des Testsatzes.
1. Führen Sie eine Fehleranalyse unseres Modells durch.

Wir können ein einfaches Modell erstellen, das aus verschiedenen Schichten besteht, wie zum Beispiel:
* Conv2D: (32 Filter der Größe 3 x 3) Die Features werden aus dem Bild „extrahiert“.
* MaxPooling2D: Die Bilder werden halbiert.
* Flatten: Transformiert das Format der Bilder von einem 2D-Array in ein 1D-Array mit 150 150 3 Pixelwerten.
* Relu: Wenn ein Wert x gegeben ist, wird max(x, 0) zurückgegeben.
* Softmax: 6 Neuronen, Wahrscheinlichkeit, dass das Bild zu einer der Klassen gehört.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

Dann können wir es mit einigen Parametern kompilieren, wie zum Beispiel:
* **Optimierer**: adam = RMSProp + Momentum.
Was ist Momentum und RMSProp?
* Momentum = berücksichtigt den Verlauf der Vergangenheit, um eine bessere Aktualisierung zu ermöglichen.
* RMSProp = exponentiell gewichteter Durchschnitt der Quadrate vergangener Gradienten.
* **Verlustfunktion**: Wir verwenden zur Klassifizierung eine spärliche kategoriale Kreuzentropie, jedes Bild gehört nur zu einer Klasse

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

Wir passen das Modell an die Daten aus dem Trainingssatz an. Das neuronale Netzwerk lernt das Muster selbst, um die einzelnen Kategorien zu unterscheiden.

In [ ]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)

In [ ]:
def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['acc'],'bo--', label = "acc")
    plt.plot(history.history['val_acc'], 'ro--', label = "val_acc")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_accuracy_loss(history)

We should evaluate the model performance on test set

In [ ]:
test_loss = model.evaluate(test_images, test_labels)

We see that we achieve 0.76 accuracy on the testing test. We got a slight underfitting :(

Let's see how the classifier is doing on random images.

In [ ]:
predictions = model.predict(test_images)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability

display_random_image(class_names, test_images, pred_labels)

## Fehleranalyse

Wir können versuchen zu verstehen, bei welcher Art von Bildern der Klassifikator Probleme hat.

In [ ]:
def print_mislabeled_images(class_names, test_images, test_labels, pred_labels):
    """
        Print 25 examples of mislabeled images by the classifier, e.g when test_labels != pred_labels
    """
    BOO = (test_labels == pred_labels)
    mislabeled_indices = np.where(BOO == 0)
    mislabeled_images = test_images[mislabeled_indices]
    mislabeled_labels = pred_labels[mislabeled_indices]

    title = "Some examples of mislabeled images by the classifier:"
    display_examples(class_names,  mislabeled_images, mislabeled_labels)



In [ ]:
print_mislabeled_images(class_names, test_images, test_labels, pred_labels)

In [ ]:
CM = confusion_matrix(test_labels, pred_labels)
ax = plt.axes()
sn.heatmap(CM, annot=True, 
           annot_kws={"size": 10}, 
           xticklabels=class_names, 
           yticklabels=class_names, ax = ax)
ax.set_title('Confusion matrix')
plt.show()

## Fazit: Der Klassifikator hat Probleme mit zwei Arten von Bildern.
Es gibt Probleme mit Straßen und Gebäuden. Nun, es kann verständlich sein, da es Gebäude auf der Straße gibt.
Es gibt auch Probleme mit dem Meer, dem Gletscher und den Bergen. Es fällt mir schwer, sie vollständig zu unterscheiden.
Es kann jedoch Wälder sehr genau erkennen!

**Zwischenaktualisierung Januar 2020**

* Merkmalsextraktion mit VGG16, trainiert auf ImageNet


* Ensemble-Modelle neuronaler Netze mit den aus VGG extrahierten Merkmalen

Inspiriert von: https://machinelearningmastery.com/model-averaging-ensemble-for-deep-learning-neural-networks/

* Feinabstimmung mit VGG16, trainiert auf ImageNet

# Merkmalsextraktion mit VGG ImageNet

Wir können Funktionen aus VGG16 extrahieren.

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=False)

Holen Sie sich die Funktionen direkt von VGG16

In [ ]:
train_features = model.predict(train_images)
test_features = model.predict(test_images)

## Visualisieren Sie die Funktionen durch PCA

In [ ]:
n_train, x, y, z = train_features.shape
n_test, x, y, z = test_features.shape
numFeatures = x * y * z

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA(n_components = 2)

X = train_features.reshape((n_train, x*y*z))
pca.fit(X)

C = pca.transform(X) # Darstellung von Personen in den neuen Achsen
C1 = C[:,0]
C2 = C[:,1]

In [ ]:
### Figures

plt.subplots(figsize=(10,10))

for i, class_name in enumerate(class_names):
    plt.scatter(C1[train_labels == i][:1000], C2[train_labels == i][:1000], label = class_name, alpha=0.4)
plt.legend()
plt.title("PCA Projection")
plt.show()

Dank dieser PCA können wir Cluster identifizieren. Die Cluster entsprechen mehr oder weniger den Beschriftungen.

Wir sehen, dass Gletscher- und Bergpunkte sehr nahe beieinander liegen, da VGG sie als sehr ähnlich ansieht.

Wir sehen, dass es keinen Unterschied zwischen Gebäude und Straße gibt.

## Training zusätzlich zu VGG

Trainieren wir ein einfaches einschichtiges neuronales Netzwerk anhand der aus VGG extrahierten Funktionen.

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (x, y, z)),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

model2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

history2 = model2.fit(train_features, train_labels, batch_size=128, epochs=15, validation_split = 0.2)

In [ ]:
plot_accuracy_loss(history)

1. Wir sollten eine Genauigkeit von ungefähr 0,844 (+0,1 Genauigkeit) gegenüber dem einfachen ConvNet erreichen.

In [ ]:
test_loss = model2.evaluate(test_features, test_labels)

# Ensemble neuronaler Netze

In [ ]:
np.random.seed(seed=1997)
# Number of estimators
n_estimators = 10
# Proporition of samples to use to train each training
max_samples = 0.8

max_samples *= n_train
max_samples = int(max_samples)

Wir definieren n_estimators Neuronale Netze.

Jedes neuronale Netzwerk wird anhand zufälliger Teilmengen des Trainingsdatensatzes trainiert. Jede Teilmenge enthält max_samples-Beispiele.

In [ ]:
models = list()
random = np.random.randint(50, 100, size = n_estimators)

for i in range(n_estimators):
    
    # Modell
    model = tf.keras.Sequential([ tf.keras.layers.Flatten(input_shape = (x, y, z)),
                                # One layer with random size
                                    tf.keras.layers.Dense(random[i], activation=tf.nn.relu),
                                    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
                                ])
    
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Store-Modell
    models.append(model)

In [ ]:
histories = []

for i in range(n_estimators):
# Trainieren Sie jedes Modell anhand einer Tasche mit Trainingsdaten
    
    train_idx = np.random.choice(len(train_features), size = max_samples)
    histories.append(models[i].fit(train_features[train_idx], train_labels[train_idx], batch_size=128, epochs=10, validation_split = 0.1))

Wir aggregieren die einzelnen Vorhersagen jedes Modells, um eine endgültige Vorhersage zu erstellen.

In [ ]:
predictions = []
for i in range(n_estimators):
    predictions.append(models[i].predict(test_features))
    
predictions = np.array(predictions)
predictions = predictions.sum(axis = 0)
pred_labels = predictions.argmax(axis=1)

Wir sollten unser Ergebnis verbessern, da wir eine geringere Varianz haben.

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy : {}".format(accuracy_score(test_labels, pred_labels)))

# Feinabstimmung von VGG ImageNet

In [ ]:
from keras.models import Model

model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=model.inputs, outputs=model.layers[-5].output)

In [ ]:
train_features = model.predict(train_images)
test_features = model.predict(test_images)

In [ ]:
from keras.layers import Input, Dense, Conv2D, Activation , MaxPooling2D, Flatten

model2 = VGG16(weights='imagenet', include_top=False)

input_shape = model2.layers[-4].get_input_shape_at(0) # get the input shape of desired layer
layer_input = Input(shape = (9, 9, 512)) # a new input tensor to be able to feed the desired layer
# https://stackoverflow.com/questions/52800025/keras-give-input-to-intermediate-layer-and-get-final-output

x = layer_input
for layer in model2.layers[-4::1]:
    x = layer(x)
    
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(100,activation='relu')(x)
x = Dense(6,activation='softmax')(x)

# Erstellen Sie das Modell
new_model = Model(layer_input, x)

In [ ]:
new_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
new_model.summary()

In [ ]:
history = new_model.fit(train_features, train_labels, batch_size=128, epochs=10, validation_split = 0.2)

In [ ]:
plot_accuracy_loss(history)

In [ ]:
from sklearn.metrics import accuracy_score

predictions = new_model.predict(test_features)    
pred_labels = np.argmax(predictions, axis = 1)
print("Accuracy : {}".format(accuracy_score(test_labels, pred_labels)))